In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import os
from os.path import join
import logging
import json

import pandas as pd
from gridemissions.load import GraphData
from gridemissions.eia_api_v1 import BAs, KEYS, SRC
import gridemissions
import re
import numpy as np

In [ ]:
from gridemissions.viz import set_plots

COLORS, PAGE_WIDTH, ROW_HEIGHT = set_plots()

from gridemissions.viz.reports import cleaning_plot, summ_stats
from matplotlib.backends.backend_pdf import PdfPages

from gridemissions.papers.physics_informed import figure1, figure2, figure3, figure4

In [ ]:
print(gridemissions.config["APEN_PATH"])
FIG_PATH = join(gridemissions.config["APEN_PATH"], "figures")
os.makedirs(FIG_PATH, exist_ok=True)

# Load data

In [ ]:
file_name = join(gridemissions.config["APEN_PATH"], "data", "EBA_%s.csv")

raw = GraphData(fileNm=file_name % "raw")
basic = GraphData(fileNm=file_name % "basic")
rolling = GraphData(fileNm=file_name % "rolling")
opt = GraphData(fileNm=file_name % "opt")
opt_no_src = GraphData(fileNm=file_name % "opt_no_src")

# Figure 1: cleaning procedure explained

In [ ]:
start, end = (pd.to_datetime("20181101"), pd.to_datetime("20181107"))
ba_data1 = GraphData(df=raw.df.loc[start:end, :])
ba_data2 = GraphData(df=opt.df.loc[start:end, :])
ba = "SOCO"

f, ax = figure1(
    ba,
    GraphData(df=raw.df.loc[start:end]),
    GraphData(df=basic.df.loc[start:end]),
    GraphData(df=rolling.df.loc[start:end]),
    GraphData(df=opt.df.loc[start:end]),
)

f.savefig(join(FIG_PATH, "fig1.pdf"))

In [ ]:
start, end = (pd.to_datetime("20181101"), pd.to_datetime("20181130"))
_ = figure1(
    "MISO",
    GraphData(df=raw.df.loc[start:end]),
    GraphData(df=basic.df.loc[start:end]),
    GraphData(df=rolling.df.loc[start:end]),
    GraphData(df=opt.df.loc[start:end]),
    save_fig=False,
)

# Figure 2: Summary results on cleaning (post July 2018)

In [ ]:
deltas = opt.df - raw.df
delta_quantiles = deltas.quantile([0.1, 0.5, 0.9])
deltas_abs = deltas.abs().quantile([0.1, 0.5, 0.9])
median_raw = raw.df.abs().quantile(0.5)
delta_quantiles_pct = delta_quantiles / median_raw * 100
deltas.loc["20180701":].isna().sum() / len(deltas)

In [ ]:
n = 13
ba_list = (
    raw.df.loc[:, [c for c in raw.get_cols(raw.regions, "D") if c in raw.df.columns]]
    .quantile(0.5)
    .sort_values(ascending=False)
    .index[0 : n * 3]
)
ba_list = [re.split(r"\.|-|_", ba)[1] for ba in ba_list]
ba_list.reverse()
print(f"Making figure for {len(ba_list)} regions")

f, ax = figure2(ba_list, n, delta_quantiles, median_raw, raw)
f.savefig(join(FIG_PATH, "fig2.pdf"))

ba_list.reverse()

# Figures 3 and 4
Normalized by TI for that BA

In [ ]:
# Do one plot for WECC BAs and one for EIC BAs
WECC_BAs = [
    "AVA",
    "AZPS",
    "BANC",
    "BPAT",
    "CHPD",
    "CISO",
    "DEAA",
    "DOPD",
    "EPE",
    "GCPD",
    "GRMA",
    "GWA",
    "HGMA",
    "IID",
    "IPCO",
    "LDWP",
    "NEVP",
    "NWMT",
    "PACE",
    "PACW",
    "PGE",
    "PNM",
    "PSCO",
    "PSEI",
    "SCL",
    "SRP",
    "TEPC",
    "TIDC",
    "TPWR",
    "WACM",
    "WALC",
    "WAUW",
    "WWA",
]
# figure3(ba_list[0:21], deltas_abs, median_raw)
sorted_WECC_BAs = [r for r in ba_list if r in WECC_BAs]
sorted_not_WECC_BAs = [r for r in ba_list if r not in WECC_BAs]
f, ax = figure3(sorted_WECC_BAs, deltas_abs, median_raw, raw)
f.savefig(join(FIG_PATH, "fig3a.pdf"))

f, ax = figure3(sorted_not_WECC_BAs, deltas_abs, median_raw, raw)
f.savefig(join(FIG_PATH, "fig3b.pdf"))

# Figure 5: Generation by source

In [ ]:
nr = 13
f, ax = figure4(ba_list[0 : 3 * nr], nr, deltas_abs, median_raw, raw)
f.savefig(join(FIG_PATH, "fig4.pdf"))

# BA report 1 on final dataset (also need to add back consumption emissions?)

In [ ]:
ba_data1 = GraphData(df=opt.df.loc[:, :])

pdf_handler = PdfPages(join(FIG_PATH, "si_report_after.pdf"))
for ba in ba_list:
    f, axes = cleaning_plot(
        ba_data1, ba, w_id=True, scale=1e-3, w_balance=True, add_title=r"after"
    )
    pdf_handler.savefig(f)
    plt.close()
pdf_handler.close()

# BA report 2 on deltas (:=opt - raw)

In [ ]:
ba_data1 = GraphData(df=opt.df.loc[:, :] - raw.df.loc[:, :])
pdf_handler = PdfPages(join(FIG_PATH, "si_report_delta.pdf"))
for ba in ba_list:
    f, axes = cleaning_plot(
        ba_data1,
        ba,
        w_id=True,
        scale=1e-3,
        w_balance=False,
        add_title=r"(After - Before)",
    )
    pdf_handler.savefig(f)
    plt.close()
pdf_handler.close()

# BA report 3 on raw data

In [ ]:
ba_data1 = GraphData(df=raw.df.loc["20180701":, :])
pdf_handler = PdfPages(join(FIG_PATH, "si_report_raw.pdf"))
for ba in ba_list:
    f, axes = cleaning_plot(
        ba_data1, ba, w_id=True, scale=1e-3, w_balance=True, add_title=r"before"
    )
    pdf_handler.savefig(f)
    plt.close()
pdf_handler.close()